In [22]:
import os
from Bio import SeqIO
import io
from Bio.Seq import Seq
import subprocess

In [23]:
blast_output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/2.Blast/1.Blast_output"
species = "Pararge_aegeria"

annotated_genome_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species"
query_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/2.Blast/0.Query"
query_species = "5.Bicyclus_anynana"
query_transcript = "XM_024088150.2"

genome_location = "/mnt/f/Genomes_2023-12-26"


In [82]:
output_coordinate_file = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length," +  "AG_GT," + "Spliceator_prediction\n"

list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
coordinate_file_name = ''
for file_names in list_of_files_in_species_folder:
    if file_names.endswith("_coordinates.csv"):
        coordinate_file_name = file_names
if coordinate_file_name =='':
    print(f"Coordinate file error")
    assert False

with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
    coordinate_file_lines = temp_file_open.readlines()
# print(coordinate_file_lines[0])

for i in range(1,len(coordinate_file_lines)):
    coordinate_file_lines_split = coordinate_file_lines[i].split(",")
    current_exon_coordinates = coordinate_file_lines[i].split(",")
    if coordinate_file_lines_split[5] == "Y":
        if i == 1:
            print(f'First Exon has errors\n{coordinate_file_lines[i]}')
            assert False
        previous_exon_number = i-1
        next_exon_number = i+1
        previous_exon_coordinates = ''
        next_exon_coordinates = ''
        while True:
            if coordinate_file_lines[previous_exon_number].split(",")[5] == "N":
                previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                break
            else:
                proceed_prompt = input(f"Previous exon coordinate\n{coordinate_file_lines[previous_exon_number]}\nProceed?")
                if proceed_prompt.lower()[0] == "y":
                    previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                    break
            previous_exon_number -= 1
        while True:
            if coordinate_file_lines[next_exon_number].split(",")[5] == "N":
                next_exon_coordinates = coordinate_file_lines[next_exon_number]
                break
            else:
                proceed_prompt = input(f"Previous exon coordinate\n{coordinate_file_lines[next_exon_number]}\nProceed?")
                if proceed_prompt.lower()[0] == "y":
                    next_exon_coordinates = coordinate_file_lines[next_exon_number]
                    break
            next_exon_number += 1

        
        print(previous_exon_coordinates)
        print(next_exon_coordinates)
        annotated_species_name, error_exon, species, genome_file, gene_sequence,left_overhang,right_overhang,scaffold,original_query_name = mafft_process(previous_exon_coordinates,
                                                                                                          next_exon_coordinates,
                                                                                                          current_exon_coordinates,
                                                                                                          query_species,
                                                                                                          query_transcript,
                                                                                                          query_location,
                                                                                                          annotated_genome_location,
                                                                                                             genome_location)

        mafft_run_folder = run_mafft(annotated_genome_location,annotated_species_name,error_exon)

        mafft_run_folder = f'{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment'
        possible_gene_sequence = process_mafft_output(mafft_run_folder)
        
        while True:
            print(f"New query = {possible_gene_sequence}")
            check_gene_sequence = input("is the query sequence ok?")
            if check_gene_sequence.lower()[0] == "n":
                possible_gene_sequence = input("Enter the desired query sequence :")
                break
            if check_gene_sequence.lower()[0] == "y":
                break
            
        run_blast_with_new_query(annotated_genome_location,annotated_species_name,error_exon,possible_gene_sequence,genome_location,species, genome_file, gene_sequence)
        start_coordinate,stop_coordinate,gt_ag, splice_prediction = process_genome_fragment_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold,original_query_name  )
        new_coordinate_file_line = process_genome_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold, gt_ag, splice_prediction,original_query_name )    
        output_coordinate_file += new_coordinate_file_line
        # break
    else:
        output_coordinate_file +=coordinate_file_lines[i].rstrip()+",NA,NA\n"
        # break
        # print(coordinate_file_lines[i])
    print(output_coordinate_file)



Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length,AG_GT,Spliceator_prediction
Pararge_aegeria,NC_053208.1,16934367,16934455,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_1,1,29,29,NA,NA

Pararge_aegeria,NC_053208.1,16934367,16934455,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_1,1,29,29

Pararge_aegeria,NC_053208.1,16926960,16927135,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_3,1,57,58

Getting Gene
min = set152_frame2, 0.443
set152_frame2
HSSKSTHSESNSSGSSGYGGKPSSSGY
New query = HSSKSTHSESNSSGSSGYGGKPSSSGY


is the query sequence ok? y




Building a new DB, current time: 02/17/2024 16:59:07
New DB name:   /mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species/33.Pararge_aegeria/Period_gene_genomic_sequence_individual_exon/gene_sequence_all.fa
New DB title:  gene_sequence_all.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.00122714 seconds.
[[1, 2]]
reached here
sequence:
CACAGCTCCAAATCTACTCATTCCGAGAGCAACTCTAGTGGAAGCAGTGGATATGGCGGAAAACCGTCCTCATCTGGATACAG
left = AG, right =Gt
HSSKSTHSESNSSGSSGYGGKPSSSGY
[[1, 2]]
start_coordinate : 16927624,stop_coordinate : 16927544
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length,AG_GT,Spliceator_prediction
Pararge_aegeria,NC_053208.1,16934367,16934455,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_1,1,29,29,NA,NA
Pararge_aegeria,NC_053208.1,16927542,16927625,1,N,5.Bicyclus_anynana_XM_024088150.2_Frame_1_rightoh_2_query_Exon_2,1,27,27,Y

KeyboardInterrupt: Interrupted by user

In [80]:
possible_gene_sequence = "PNSPVGTIQYVSSLPNQNNVS"
run_blast_with_new_query(annotated_genome_location,annotated_species_name,error_exon,possible_gene_sequence,genome_location,species, genome_file, gene_sequence)
start_coordinate,stop_coordinate,gt_ag, splice_prediction = process_genome_fragment_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold,original_query_name  )
new_coordinate_file_line = process_genome_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold, gt_ag, splice_prediction,original_query_name )    
output_coordinate_file += new_coordinate_file_line



Building a new DB, current time: 02/17/2024 16:50:20
New DB name:   /mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species/33.Pararge_aegeria/Period_gene_genomic_sequence_individual_exon/gene_sequence_all.fa
New DB title:  gene_sequence_all.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.00235009 seconds.
[[0, 0]]
reached here
sequence:
CCCAACTCACCGGTTGGCACCATACAGTATGTCTCCAGCTTACCAAACCAAAATAATGTGAGT
left = GT, right =AT
sequence:
TCCCAACTCACCGGTTGGCACCATACAGTATGTCTCCAGCTTACCAAACCAAAATAATGTGAGTA
left = TG, right =TA
sequence:
GTCCCAACTCACCGGTTGGCACCATACAGTATGTCTCCAGCTTACCAAACCAAAATAATGTGAGTAT
left = TT, right =AA
sequence:
TGTCCCAACTCACCGGTTGGCACCATACAGTATGTCTCCAGCTTACCAAACCAAAATAATGTGAGTATA
left = CT, right =AG
sequence:
TTGTCCCAACTCACCGGTTGGCACCATACAGTATGTCTCCAGCTTACCAAACCAAAATAATGTGAGTATAA
left = TC, right =GT
sequence:
CTTGTCCCAACTCACCGGTTGGCACCATACAGTATGTCTCCAGCTTACCAA

In [79]:
print(output_coordinate_file)

Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length,AG_GT,Spliceator_prediction
Pararge_aegeria,NC_053208.1,16934367,16934455,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_1,1,29,29,NA,NA
Pararge_aegeria,NC_053208.1,16927542,16927625,1,N,5.Bicyclus_anynana_XM_024088150.2_Frame_1_rightoh_2_query_Exon_2,1,27,27,Y,Y
Pararge_aegeria,NC_053208.1,16926960,16927135,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_3,1,57,58,NA,NA
Pararge_aegeria,NC_053208.1,16925888,16925976,1,N,5.Bicyclus_anynana_XM_024088150.2_query_Exon_4,4,28,28,NA,NA
Pararge_aegeria,NC_053208.1,16925720,16925794,1,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_0_query_Exon_5,1,25,25,Y,Y



In [180]:
possible_gene_sequence = process_mafft_output(mafft_run_folder)
check_gene_sequence = input("is the query sequence ok?")
if check_gene_sequence == "n":
    gene_sequence = input("Enter the desired query sequence :")


run_blast_with_new_query(annotated_genome_location,annotated_species_name,error_exon,possible_gene_sequence,genome_location,species, genome_file, gene_sequence)


min = set152_frame2, 0.443
set152_frame2
HSSKSTHSESNSSGSSGYGGKPSSSGY
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species/33.Pararge_aegeria/Period_gene_genomic_sequence_individual_exon
Exon_2
Spliceator_results(1).csv
all temp
desktop.ini
gene_sequence_all.fa
gene_sequence_all.fa.nhr
gene_sequence_all.fa.nin
gene_sequence_all.fa.nsq
temp


Building a new DB, current time: 02/17/2024 03:20:35
New DB name:   /mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species/33.Pararge_aegeria/Period_gene_genomic_sequence_individual_exon/gene_sequence_all.fa
New DB title:  gene_sequence_all.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.00960398 seconds.


In [183]:
start_coordinate,stop_coordinate,gt_ag, splice_prediction = process_genome_fragment_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold )
process_genome_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold, gt_ag, splice_prediction )    

[[1, 2]]
[[1, 2]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length,AG_GT,Spliceator_prediction

33.Pararge_aegeria,NC_053208.1,16927542,16927625,1,N,Query_Exon_2,1,27,27,Y,Y



In [65]:
def mafft_process(previous_exon_coordinates,
                  next_exon_coordinates,
                  current_exon_coordinates,
                  query_species,
                  query_transcript,
                  query_location,
                  annotated_genome_location,
                 genome_location):
    query_exon = current_exon_coordinates[6].split("query")[-1][1:]
    error_exon = query_exon
    # print("AA")
    genome_file = get_genome_file(genome_location,species)
    
    upstream_exon_line, downstream_exon_line = previous_exon_coordinates.split(","),next_exon_coordinates.split(",")
    complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]

    if complement == "0":
        gene_start = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
        gene_end = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
    
    if complement == "1":
        gene_start = max(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        gene_end = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
    # print("BB")

    # print(gene_sequence)
    # gene_sequence = Seq("GGTAAggacattataatttttaatgcatTCCCTCGAAACTTGAAATCTTCTAATCCACTAgagtttctataaaaaaaaatatggcataTTATGCCGTCATGCCAATCCATTATGGCggatatccatactaatattatgctTGTGAAATTGTttctgtttgtttctttgtttgttacctatgaaCAGCTAAACCGTTgaatcgattttcatgaaatttgacACACACGTAGTTTGCATCTTTAGTACAGTGAtttcattataacatttatattggcTGTAATATAAAAAGCGTTAACCATCTGATTTGTGAGATGGTGGAGCTAAGGCTAAAAATTCAGTATGTAACtattttaagttaaatgtaTAGCATCATTTGCATCTGAAGCCGTTTCATCATATAGATTGATTagcgttttttattatatttggtcGGAGTAATACTAATAATCTATTAGCGTGAGCAACATTAATTAATCTGTTTGacagattttattaaaatttactttcatGATATTGAACTATAGAAAGAGTAAAGGTGAAATAGAGAAGTTAGTAATACAAAGTGAATTCACAAAAccgttttattttacaaaaaattgaatgaaattataatttattttcatatttacagGATCGCAAAACCTTTGCATCCCAAATCACTAGCGGGCTTTTAGCTCCGAAATCAACTAGCGGTGAACAACCTCAAGGTAATTTCGATTGATTTAATGATCCATGCTTCCAAtagatttaatttgaaaatttgtcgTTTGTATTAGCTTCAATACATCACATTCTCAAACATGACTTCATACAACCAAACTTATCCgctacaattttataattgagATAGTGAAAAAGTTAATAGCTTCAAGATACAAATTTTACAGGGTTCTTTCATACAAACGTGTAATTAATTCTAAATGTTAtaactaagtataaaaataatgtttcctTTAAAGTGAACTGTCTTCTTGCTGATTCTAGTACTAGTATCaataatttcaattaataagTACCTCAATAAGATAAACCGCCATATTTTATTGCCGGCTGTTTCATgtaaaattatcttatttaaaaacatttaaaagacAAATTTACAAAATGTATTAAGCTTCTGTGAGAATTATGTGCACTATGTTTCATACATAGCAAAAGTTGTATTTCATTATTTGagatgaaattgaaaatttaatgttaataatttttgttttataatgcaTTTAAAGTAATCATCAACACATAACCTAACTTTTGGGATAGTTAAAATTTGTGTCGCAACTTCAATAAGAGCTTGTCCAC")

    with open(f"{query_location}/{query_species}/{query_transcript}/query_{query_exon}.fa", 'r') as query_file:
        query_file_list = query_file.readlines()
        query_fasta_sequence = "".join(query_file_list)
        left_overhang = query_file_list[0].split("Frame")[1][1]
        right_overhang = query_file_list[0].split("rightoh")[1][1]
        original_query_name = query_file_list[0]
        
    query_length = len(query_fasta_sequence.split("\n")[1])
    
    annotated_species_name = get_annotated_genome_name(annotated_genome_location, species)
    # print(annotated_species_name)

    # print(annotated_genome_location)
    gene_sequence = get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name)
    # assert False
    check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon)
    
    
    make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length)
    
    return(annotated_species_name, error_exon,species, genome_file,gene_sequence,left_overhang,right_overhang,scaffold, original_query_name )


In [6]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)   

In [7]:
# def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement):
#     from Bio import SeqIO
#     genome = SeqIO.parse(f"{genome_location}/{species}/{genome_file}", "fasta")
#     for entries in genome:
#         # print(scaffold)
#         if scaffold == entries.id:
#             print(scaffold)
#             gene_sequence = entries.seq[gene_start-1:gene_end]
#             if complement == "1":
#                 gene_sequence = gene_sequence.reverse_complement()
#             break
#     return (gene_sequence)

def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name):
    from Bio import SeqIO
    print("Getting Gene")
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if "Period_gene_genomic_sequence_individual_exon" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")

    list_of_files_inside_indiv_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")
    if "temp" not in list_of_files_inside_indiv_exon_folder:
         os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp")

    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}"')
    # subprocess.run("pwd")
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa"')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    genome = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    return (gene_sequence)

In [8]:
def get_annotated_genome_name(annotated_genome_location, species):
    list_of_annotated_genomes = os.listdir(annotated_genome_location)
  
    # print(annotated_species)
    for annotated_species in list_of_annotated_genomes:
        if annotated_species.endswith(species):
            return(annotated_species)
             
    if annotated_species_name == '':
        print(f"Error with annotated species name")
        assert False


In [9]:
def check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon):
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if "Period_gene_genomic_sequence_individual_exon" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")
    
    list_of_exon_directories = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/")
    
    if error_exon not in list_of_exon_directories:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}")
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment")
    elif error_exon in list_of_exon_directories:
        list_of_folders_1 = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}")
        if "for_alignment" not in list_of_folders_1:
            os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment")
        list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment")
        for file in list_of_files:
            os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment/{file}")


In [10]:
def make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length):
    for offset in range(3):
        translated_sequence = str(gene_sequence[offset:].translate()).split("*")
        for i in range(len(translated_sequence)):
            if len(translated_sequence[i])> 0.8*query_length:
                sequence_set = f">set{i+1}_frame{offset}\n{translated_sequence[i]}\n\n"
                # print(i+1, offset)
                
                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment/{error_exon}_translated_genomic_sequence_{i+1}_frame{offset}.fa",'w') as out_file:
                    output = f"{query_fasta_sequence}\n\n{sequence_set}"
                    out_file.write(output)

In [11]:
def run_mafft(annotated_genome_location,annotated_species_name,error_exon):
    location = f'{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_alignment'
    list_of_files_to_run_mafft_on = os.listdir(location)
    for file in list_of_files_to_run_mafft_on:
        if file.endswith(".fa"):
            command = f'"/home/saurav/miniconda3/envs/ncbi_datasets/bin/mafft" --localpair --maxiterate 16 --reorder --distout "{location}/{file}" > "{location}/alignment_{file}.txt"'
            # print(command)
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            # os.system(f'{command}')
            command = f'"/home/saurav/miniconda3/envs/ncbi_datasets/bin/mafft" --localpair --clustalout --maxiterate 16 --reorder "{location}/{file}" > "{location}/alignment_clustal_{file}.txt"'
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            
            # os.system(f'{command}')
    return(location)

In [12]:
def process_mafft_output(mafft_run_folder):
    list_of_files_in_mafft_run_folder = os.listdir(mafft_run_folder)
    score_output = ''
    score = 99
    min_score_sequence = ''
    alignment_file = ''
    for file in list_of_files_in_mafft_run_folder:
        if file.endswith(".fa.hat2"):
            with io.open(f"{mafft_run_folder}/{file}", 'r') as dist_matrix_file:
                dist_matrix_list = dist_matrix_file.readlines()
            
            sequence_name = dist_matrix_list[-2].rstrip().split("=")[1]
            distance_score = float(dist_matrix_list[-1].rstrip())
            score_output += f"{sequence_name} =  {distance_score}\n"
            if distance_score < score:
                score = distance_score
                min_score_sequence = sequence_name
                alignment_file = f'alignment_{file.replace(".hat2",".txt")}'
    # print(score_output)
    print(f"min = {min_score_sequence}, {score}" )
    
    alignment_file = SeqIO.parse(f"{mafft_run_folder}/{alignment_file}", 'fasta')
    # print (records.id)
    fasta_start_position = 0
    fasta_end_position = 0
    start_switch = 0
    end_switch = 0
    for records in alignment_file:
        
        if start_switch == 1 and end_switch == 1:
            gene_sequence = records.seq[fasta_start_position:fasta_end_position]
            print(f"{records.id}\n{gene_sequence}")
        if error_exon in records.id:
            for current_position in range(len(records.seq)):
                # print(records.seq[current_position])
                if "-" not in records.seq[current_position] and start_switch == 0:
                    fasta_start_position = current_position
                    start_switch = 1
                if end_switch == 1 and "-" not in records.seq[current_position]:
                    end_switch = 0
                    
                if start_switch == 1 and records.seq[current_position] == "-" and end_switch == 0:
                    fasta_end_position = current_position
                    end_switch = 1
    return(gene_sequence)
    


In [13]:
def run_blast_with_new_query(annotated_genome_location,
                             annotated_species_name,
                             error_exon, 
                             possible_gene_sequence,
                             genome_location, 
                             species, 
                             genome_file, 
                             gene_sequence):
    
    list_of_folders_inside_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}")
    if "for_blast" not in list_of_folders_inside_exon_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast")
    else:
        list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast")
        for file in list_of_files:
            os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast/{file}")


    
    with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast/new_query.txt",'w') as query_file:
        query = f">Query_{error_exon}\n{possible_gene_sequence}"
        query_file.write(query)

    query_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast/new_query.txt"
    genome = f"{genome_location}/{species}/{genome_file}"
    out_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast"
    # blast_command = f'tblastn -seg no -query "{query_location}" -db "{genome}" -num_alignments 3 -out "{out_location}/blast_out.htm" -html'
    # os.system(f'{blast_command}')
    # blast_command = f'tblastn -seg no -query "{query_location}" -db "{genome}" -num_alignments 3 -out "{out_location}/blast_out.txt"'
    # os.system(f'{blast_command}')

    genome_fragment_out = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast/local_genomic_fragment.fa"
    with io.open(genome_fragment_out,'w') as out_file:
        output = f">Genome_fragment_{error_exon}\n{gene_sequence}"
        out_file.write(output)

    local_genomic_fragment_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/"
    cd_command = f'cd "{local_genomic_fragment_location}"\nmakeblastdb -in gene_sequence_all.fa -dbtype nucl\n'
    # os.system(f'{cd_command}')
    subprocess.run(f'{cd_command}', shell = True, stderr = subprocess.DEVNULL)
    # print(mkdb_command)
    # os.system(f'{mkdb_command}')
    blast_command = f'cd "{out_location}"\ntblastn -seg no -query new_query.txt -db ../../gene_sequence_all.fa -num_alignments 3 -out blast_out_genome_fragment.htm -html'
    # os.system(f'{blast_command}')
    subprocess.run(f'{blast_command}', shell = True, stderr = subprocess.DEVNULL)
    blast_command = f'cd "{out_location}"\ntblastn -seg no -query new_query.txt -db ../../gene_sequence_all.fa -num_alignments 3 -out blast_out_genome_fragment.txt'
    # os.system(f'{blast_command}')
    subprocess.run(f'{blast_command}', shell = True, stderr = subprocess.DEVNULL)

    
        
# run_blast_with_new_query(annotated_genome_location,annotated_species_name,error_exon,possible_gene_sequence,genome_location,species, genome_file, gene_sequence)        

In [18]:
def process_spiceator_result(start_coordinate, stop_coordinate,annotated_genome_location,annotated_species_name):
    results_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/"
    list_of_files_here = os.listdir(results_location)
    spliceator_results_file = ''
    for files in list_of_files_here:
        if files.startswith("Spliceator_results"):
            spliceator_results_file = files
    if spliceator_results_file == '':
        print("Splice file missing")
        assert False

    with open(f"{results_location}/{spliceator_results_file}", 'r') as splice_file:
        splice_file_content = splice_file.readlines()

    splice_acceptor_presence = "N"
    splice_donor_presence = "N"
    splice_donor = ''
    splice_acceptor = ''
    for lines in splice_file_content:
        line_split = lines.split(";")
        if line_split[0]=="Acceptor" and splice_acceptor_presence == "N":
            acceptor_start = int(line_split[1])
            acceptor_end = acceptor_start + len(line_split[3])
            if start_coordinate >=acceptor_start and start_coordinate <=acceptor_end:
                splice_acceptor_presence = "Y"
                splice_acceptor = lines
        if line_split[0]=="Donor" and splice_donor_presence == "N":
            donor_start = int(line_split[1])
            donor_end = donor_start + len(line_split[3])
            if stop_coordinate >=donor_start and stop_coordinate <=donor_end:
                splice_donor_presence = "Y"
                splice_donor = lines

    return(splice_acceptor_presence, splice_donor_presence, splice_donor, splice_acceptor)
                

In [76]:
def process_genome_fragment_blast_file(annotated_genome_location, 
                                       annotated_species_name,
                                       error_exon,
                                       left_overhang,
                                       right_overhang,
                                       scaffold,
                                       original_query_name  ):
    blast_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast"
    with open(f"{blast_location}/new_query.txt", 'r') as query_file:
        query_name_list = [query_file.readlines()[0][1:].rstrip()]
        seq_modi = [[int(left_overhang),int(right_overhang)]]
        print(seq_modi)

    header = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length\n" 
    Output_Sequence = header
    scaff = "Intial_value"
    scaff_old = "Intial_value"
    old_end = 0
    species_name = annotated_species_name

    for i in range(len(query_name_list)):
        query_name = query_name_list[i]
        Length_switch = "0"
        
        with open(f"{blast_location}/blast_out_genome_fragment.txt",'r') as tblast_out:
            lines_in_file = tblast_out.readlines()

        result_section_switch = 0
        start_coor_switch = 0
        query_start_coor_switch = 0
        stop_coor_switch = 0
        error = "N"
        break_switch = 0

        start = 0
        stop = 0
        start_coor = 0
        stop_coor = 0
        query_length = 0
        gt_ag = "N"

        for lines in lines_in_file:

#             print(lines)
            if query_name in lines:
            #Initialize that results can now be checked
                result_section_switch = 1
                query_species_split = lines.split(" ")[1].split("_")
                query_species = str(query_species_split[1]+"_"+query_species_split[2].rstrip())

            if result_section_switch == 1 and "Lambda" in lines:
            #This block indicates end of the results block in blast output
                result_section_switch == 0
                
                break

            if result_section_switch == 1:
            #While checking the result
                if "Length=" in lines and Length_switch == "0":
                #Get query length from the blast output
                    
                    query_length = int(lines.split("=")[1].rstrip())
                    
                    Length_switch = 1 #Indicated length has been acquired
                    
                if ("Score" in lines or ">" in lines) and (start_coor_switch == 1):
    #                print (lines)
                    break
        
                if ">" in lines:
                #Start of the first result
                    scaff = lines.split(" ")[0][1:] #Scaffold from the result
                    if scaff_old != "Intial_value" and scaff_old != scaff:
                        error = "Y"
                    scaff_old = scaff
                    
                if "Query" in lines and "=" not in lines:
                #Read the query line in output
                    if query_start_coor_switch == 0:
#                        print(lines)
                        query_start_coor = int(lines.split(" ")[2])
                        query_start_coor_switch = 1
                        #Query start coordinate fixed
            
                    query_stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting query stop coordinates for multiline result
    #                print (stop_coor)
                    
                if "Sbjct" in lines:
                #Read the blast target line
                    if start_coor_switch == 0:
                        start_coor = int(lines.split(" ")[2])
                        start_coor_switch = 1
                    stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting target stop coordinates for multiline result
                
                

        
        if break_switch == 1:
            break

        if start_coor < stop_coor:
            complement = "0" #Forward complement
            
            length = (stop_coor-start_coor)/3
            start = start_coor
            stop = stop_coor

        elif start_coor > stop_coor:
            complement = "1" #Reverse complement
            length = (-stop_coor+start_coor)/3
            start = stop_coor
            stop = start_coor

        else:
            error = "Y"
        
        
        
        
        seq_length = query_length
        if (start != 0 or stop != 0):
            start_modifier = seq_modi[i][0]
            stop_modifier = seq_modi[i][1]
        else:
            start_modifier = 0
            stop_modifier = 0  
        #Check if the length of target (blast hit) is significantly smaller than query
        if length < query_length - 0.2*query_length:
            error = "Y"

        old_trans = ''

        if query_start_coor != "1" and query_name != query_name_list[0]:
            if complement == "0":
                start = int(start) - 3*(int(query_start_coor)-1)                
            if complement == "1":
                stop = int(stop) + 3*(int(query_start_coor)-1)
    
    #For the end
        if query_stop_coor != str(seq_length) and query_name != query_name_list[-1]:
            if complement == "0":
                stop = int(stop) + 3*(int(seq_length)-int(query_stop_coor))
            if complement == "1":
                

                start = int(start) - 3*(int(seq_length)-int(query_stop_coor))
 
        
        
        
#Adding or removing 3' and 5' overhangs for forward and reverse complement
    #For forward complement
        if complement == "0":
            start = int(start) - int(start_modifier)
            stop = int(stop) +  int(stop_modifier)
            if old_end != 0 and old_end > stop:

                error = "Y"
            old_end = stop

    #For reverse complement
        if complement == "1":
            start = int(start) - int(stop_modifier)
            stop = int(stop) +  int(start_modifier)
            if old_end != 0 and old_end < stop:
                error = "Y"
            old_end = stop

    #Simple check for lenghth
        if start == 0 or stop == 0:
            error = "Y"

        genome_file = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/gene_sequence_all.fa", 'fasta')
        print("reached here")
        for records in genome_file:
            old_start = start
            old_stop = stop
            ag = "N"
            gt = "N"
            while True:
                print(f"sequence:\n{records.seq[start+start_modifier-1:stop]}")
                print(f"left = {records.seq[start-3:start-1]}, right ={(records.seq[stop:stop + 2])}"  ), 
                if (records.seq[start-3:start-1]).lower() == "ag" and ag != "Y":
                    
                    ag = "Y"
                    
                elif ag != "Y":
                    start -= 1
                    
                if (records.seq[stop:stop + 2]).lower() == "gt" and gt != "Y":
                    gt = "Y"
                elif gt != "Y":
                    stop +=1
                if old_start - start > 1000 or stop - old_stop > 1000:
                    break
                if gt == "Y" and ag == "Y":
                    gt_ag = "Y"
                    break
        
        query_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast/new_query_spliced.txt"
        with open(query_location , 'w') as query_file_new:
            # sequence_translated = records.seq[start+start_modifier-1:stop]
            # print(sequence_translated)
            
            sequence_translated = records.seq[start+start_modifier-1:stop].translate()
            print(sequence_translated)
            output = f">{original_query_name[1:]}\n{sequence_translated}"
            query_file_new.write(output)

        genome_file = get_genome_file(genome_location,species)
        genome = f"{genome_location}/{species}/{genome_file}"
        out_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast"
    
        
        blast_command = f'tblastn -seg no -query "{query_location}" -db "{genome}" -num_alignments 3 -out "{out_location}/blast_out.htm" -html'
        # print(blast_command)
        # subprocess.run(f'{blast_command}', shell = True, stderr = subprocess.DEVNULL)
        subprocess.run(f'{blast_command}', shell = True, stderr = subprocess.DEVNULL)
    
        blast_command = f'tblastn -seg no -query "{query_location}" -db "{genome}" -num_alignments 3 -out "{out_location}/blast_out.txt"'
        subprocess.run(f'{blast_command}', shell = True, stderr = subprocess.DEVNULL)
    

                
        acceptor, donor, don_line, acc_line = process_spiceator_result(start, stop,annotated_genome_location,annotated_species_name)

        if acceptor == "Y" and donor  == "Y":
            splice_prediction = "Y"
            
        return(start,stop,gt_ag, splice_prediction) 
        # output_format = str(species_name.split("\n")[0])+"," + str(scaffold) +"," + str(start)+"," + str(stop)+"," + str(complement)+"," + str(error)+  ","+ str(query_name)+","+ str(query_start_coor)+","+str(query_stop_coor)+","+str(query_length)+ "\n"  
        # print(output_format)



In [62]:
start_coordinate,stop_coordinate,gt_ag, splice_prediction = process_genome_fragment_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold,original_query_name  )
new_coordinate_file_line = process_genome_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold, gt_ag, splice_prediction )    
output_coordinate_file += new_coordinate_file_line

[[1, 2]]
reached here
left = AG, right =Gt
HSSKSTHSESNSSGSSGYGGKPSSSGY
[[1, 2]]
TBLASTN 2.9.0+

query_spliced


query_spliced


query_spliced
Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro A.

query_spliced
Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J.

query_spliced
Lipman (1997), "Gapped BLAST and PSI-BLAST: a new generation of

query_spliced
protein database search programs", Nucleic Acids Res. 25:3389-3402.

query_spliced


query_spliced


query_spliced


query_spliced
Database: /mnt/j/Genomes_2023-12-

query_spliced
26/Pararge_aegeria/GCF_905163445.1_ilParAegt1.1_genomic.fna

query_spliced
           70 sequences; 516,571,003 total letters

query_spliced


query_spliced


query_spliced


query_spliced
Query= query_spliced

query_spliced


IndexError: list index out of range

In [20]:
def sequence_extractor(Species, Scaff, reverse_c, start, end, frame = 1, trans = 1 ):
    from Bio import SeqIO
    from Bio.Seq import Seq
    import os
    
    out_Seq = ''
    
    
    #Genome folder
    entries = os.listdir("F:/Genomes_2023-12-26/"+Species)
    
    #Get genome file from Genome folder
    for file_names  in entries:
        if ".nhr" in file_names:
            Genome_name = file_names[:-4]
            break
    
    #Read the genome file
    fasta_file = open(("F:/Genomes_2023-12-26/"+Species+"/"+Genome_name),'r')
    
    #Extract the sequence 
    for record in SeqIO.parse(fasta_file,"fasta"):
        if record.id == Scaff:
            sequence = str(record.seq)
            out_Seq = Seq(sequence[start-1:end])
            if reverse_c == 1:
                out_Seq = out_Seq.reverse_complement()
                
    if len(out_Seq) < 10000: #fixing error due to mistake in typing the coordinate, change this for longer sequence
#        print (out_Seq)
        if frame == 1 :
            out_trans = out_Seq[0:]
        if frame == 2 :
            out_trans = out_Seq[1:]
        if frame == 3 :
            out_trans = out_Seq[2:]
    else:
        print ("too long")
        assert(False)
        
        
    #    break
    fasta_file.close()
    if trans == 1:
        return (out_trans.translate())
    else:
        return(out_Seq)

In [81]:
def process_genome_blast_file(annotated_genome_location, annotated_species_name,error_exon,left_overhang,right_overhang,scaffold, ag_gt, splice_prediction,original_query_name ):
    blast_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/{error_exon}/for_blast"
    with open(f"{blast_location}/new_query_spliced.txt", 'r') as query_file:
        query_name_list = [query_file.readlines()[0][1:].rstrip()]
        seq_modi = [[int(left_overhang),int(right_overhang)]]
        print(seq_modi)

    header = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length," +  "AG_GT," + "Spliceator_prediction\n"
    
    Output_Sequence = header
    scaff = "Intial_value"
    scaff_old = "Intial_value"
    old_end = 0
    species_name = annotated_species_name

    for i in range(len(query_name_list)):
        query_name = query_name_list[i]
        Length_switch = "0"
        
        with open(f"{blast_location}/blast_out.txt",'r') as tblast_out:
            lines_in_file = tblast_out.readlines()

        result_section_switch = 0
        start_coor_switch = 0
        query_start_coor_switch = 0
        stop_coor_switch = 0
        error = "N"
        break_switch = 0

        start = 0
        stop = 0
        start_coor = 0
        stop_coor = 0
        query_length = 0
        gt_ag = "N"

        for lines in lines_in_file:

            # print(lines)
            # print(query_name)
            if query_name in lines:
            #Initialize that results can now be checked
                result_section_switch = 1
                query_species_split = lines.split(" ")[1].split("_")
                query_species = str(query_species_split[1]+"_"+query_species_split[2].rstrip())

            if result_section_switch == 1 and "Lambda" in lines:
            #This block indicates end of the results block in blast output
                result_section_switch == 0
                
                break

            if result_section_switch == 1:
            #While checking the result
                if "Length=" in lines and Length_switch == "0":
                #Get query length from the blast output
                    
                    query_length = int(lines.split("=")[1].rstrip())
                    
                    Length_switch = 1 #Indicated length has been acquired
                    
                if ("Score" in lines or ">" in lines) and (start_coor_switch == 1):
    #                print (lines)
                    break
        
                if ">" in lines:
                #Start of the first result
                    scaff = lines.split(" ")[0][1:] #Scaffold from the result
                    if scaff_old != "Intial_value" and scaff_old != scaff:
                        error = "Y"
                    scaff_old = scaff
                    
                if "Query" in lines and "=" not in lines:
                #Read the query line in output
                    if query_start_coor_switch == 0:
#                        print(lines)
                        query_start_coor = int(lines.split(" ")[2])
                        query_start_coor_switch = 1
                        #Query start coordinate fixed
            
                    query_stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting query stop coordinates for multiline result
    #                print (stop_coor)
                    
                if "Sbjct" in lines:
                #Read the blast target line
                    if start_coor_switch == 0:
                        start_coor = int(lines.split(" ")[2])
                        start_coor_switch = 1
                    stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting target stop coordinates for multiline result
                
                

        
        if break_switch == 1:
            break
        
        print(f"start_coordinate : {start_coor},stop_coordinate : {stop_coor}")
        if start_coor < stop_coor:
            complement = "0" #Forward complement
            
            length = (stop_coor-start_coor)/3
            start = start_coor
            stop = stop_coor

        elif start_coor > stop_coor:
            complement = "1" #Reverse complement
            length = (-stop_coor+start_coor)/3
            start = stop_coor
            stop = start_coor

        else:
            error = "Y"
        
        
        
        
        seq_length = query_length
        if (start != 0 or stop != 0):
            start_modifier = seq_modi[i][0]
            stop_modifier = seq_modi[i][1]
        else:
            start_modifier = 0
            stop_modifier = 0  
        #Check if the length of target (blast hit) is significantly smaller than query
        if length < query_length - 0.2*query_length:
            error = "Y"

        old_trans = ''

        if query_start_coor != "1" and query_name != query_name_list[0]:
            if complement == "0":
                start = int(start) - 3*(int(query_start_coor)-1)                
            if complement == "1":
                stop = int(stop) + 3*(int(query_start_coor)-1)
    
    #For the end
        if query_stop_coor != str(seq_length) and query_name != query_name_list[-1]:
            if complement == "0":
                stop = int(stop) + 3*(int(seq_length)-int(query_stop_coor))
            if complement == "1":
                

                start = int(start) - 3*(int(seq_length)-int(query_stop_coor))
 
        
        
        
#Adding or removing 3' and 5' overhangs for forward and reverse complement
    #For forward complement
        if complement == "0":
            start = int(start) - int(start_modifier)
            stop = int(stop) +  int(stop_modifier)
            if old_end != 0 and old_end > stop:

                error = "Y"
            old_end = stop

    #For reverse complement
        if complement == "1":
            start = int(start) - int(stop_modifier)
            stop = int(stop) +  int(start_modifier)
            if old_end != 0 and old_end < stop:
                error = "Y"
            old_end = stop

    #Simple check for lenghth
        if start == 0 or stop == 0:
            error = "Y"

            
        output_format = str(species_name.split(".")[1])+"," + str(scaffold) +"," + str(start)+"," + str(stop)+"," + str(complement)+"," + str(error)+  ","+ str(query_name)+","+ str(query_start_coor)+","+str(query_stop_coor)+","+str(query_length)+ "," + ag_gt + "," + splice_prediction +"\n"  
        # print(Output_Sequence)
        return(output_format)